# Архитектура памяти Windows (Лабораторная работа №5, Вариант 24)
## Выполнил: студент группы БВТ2001 Хайдаров Адель Фанисович
##### Цель работы:
* изучение организации работы с виртуальной памятью в Windows NT
* получение практических навыков по использованию WinAPI для исследования памяти приложения
##### Задание: разработать программное обеспечение для приложения, которое:
* выдает информацию, получаемую при использовании API GlobalMemoryStatus, в графическом представлении.
* составляет карту виртуальной памяти для любого существующего процесса по выбору.

### Вспомогательные функции

In [2]:
open System.Runtime.InteropServices
open Microsoft.FSharp.NativeInterop
open System.Text
open System.IO
open System

open Plotly.NET

let rnd = Random()

type w = uint16
type dw = uint
type ull = uint64
type hdl = nativeint

#r "nuget: Plotly.NET.Interactive"

Installed Packages Plotly.NET.Interactive, 3.0.2

Loading extensions from `Plotly.NET.Interactive.dll`

### Функции Win32 API
Ниже представлены использованные функции Win32 API для получения той или иной информации о системе и для задания некоторых параметров

In [3]:
type Access = | Terminate = 1 | Create = 2 | Query = 0x400
type Snaps = | HeapList = 1 | Process = 2 | Thread = 4 | Module = 8 | All = 15

type MemoryState = | Committed = 0x1000 | Free = 0x10000 | Reserved = 0x2000
type Protection = | NoAccess = 0x1 | RO = 0x2 | RW = 0x4 | WCopy = 0x8
                  | Exec = 0x10 | ExR = 0x20 | ExRW = 0x40 | ExWCopy = 0x80

type Type = | Image = 0x1000000 | Mapped = 0x40000 | Private = 0x20000

[<DllImport("kernel32.dll")>] extern hdl OpenProcess(Access acc, bool inheritHandle, dw OpenProcess)

type MemoryStatus = struct
    val mutable length: dw
    val load: dw; val total: ull; val available: ull
    val pfTotal: ull; val pfAvail: ull
    val vTotal: ull; val vAvail: ull
end
[<DllImport("kernel32.dll")>] extern void GlobalMemoryStatus(MemoryStatus& status)

type MemoryInfo = struct
    val baseAddr: hdl; val allocBase: hdl
    val protection: Protection; val partition: w; val regionSize: hdl
    val state: MemoryState; val protect: dw; val pType: Type
end

[<DllImport("kernel32.dll")>] extern ull VirtualQueryEx(hdl proc, void*, MemoryInfo& info, int length)

[<DllImport("kernel32.dll")>] extern bool CloseHandle(hdl object)

[<DllImport("kernel32.dll")>] extern int GetLastError()

### Выполнение

##### Использование памяти

In [10]:
let mutable ms = MemoryStatus()
GlobalMemoryStatus &ms
display ms
let m = [ms.pfTotal; ms.pfAvail; ms.total; ms.available; ms.vTotal; ms.vAvail] 
        |> List.map (fun t -> float t / float (1 <<< 30))
let ids = ["pf"; "ph"; "ph-free"; "ph-used"]
let labels = ["Файл подкачки"; "Физическая"; "Свободно"; "Занято"]
let parents = [""; ""; "ph"; "ph"; "pf"; "pf"]
let vals = [m[0] - m[2]; m[2]; m[3]; m[2] - m[3]]
let vLabels, vVals, vParents = ["Всего"; "Доступно"; "Занято"], [m[4]; m[5]; m[4] - m[5]], [""; "Всего"; "Всего"]

load,total,available,pfTotal,pfAvail,vTotal,vAvail,length
65,8462585856,2903408640,12195880960,4630278144,140737488224256,138514004815872,56


Использование памяти:

In [11]:
Chart.Sunburst(Ids=ids, labels=labels, Values=vals, parents=parents, BranchValues=StyleParam.BranchValues.Total)
|> Chart.withTitle $"Использование памяти {ms.load}{'%'}" |> Chart.withSize(400, 450) |> Chart.withMarginSize(10, 10, 75, 40)

<!-- Plotly chart will be drawn inside this DIV -->

Виртуальное пространство:

In [12]:
Chart.Sunburst(labels=vLabels, Values=vVals, parents=vParents, BranchValues=StyleParam.BranchValues.Total)
|> Chart.withTitle "Виртуальное пространство, ГБ" |> Chart.withSize(400, 450) |> Chart.withMarginSize(10, 10, 75, 40)

<!-- Plotly chart will be drawn inside this DIV -->

##### Карты виртуальной памяти

In [13]:
let proc = OpenProcess(Access.Query, false, 9892u)

let mutable info = MemoryInfo()
let mutable rSize = 0n
let mutable over = -1n

seq {
    while rSize < (ms.vTotal |> int64 |> hdl) && over <> rSize do
        over <- rSize
        VirtualQueryEx(proc, rSize, &info, sizeof<MemoryInfo>) |> ignore
        rSize <- rSize + info.regionSize
        yield info
    proc |> CloseHandle |> ignore
}

index,baseAddr,allocBase,protection,partition,regionSize,state,protect,pType
0,0,0,0,0,531169280,Free,1,0
1,531169280,531169280,RW,0,4096,Committed,4,Private
2,531173376,0,0,0,61440,Free,1,0
3,531234816,531234816,RW,0,4096,Committed,4,Private
4,531238912,0,0,0,1616113664,Free,1,0
5,2147352576,2147352576,RO,0,4096,Committed,2,Private
6,2147356672,0,0,0,20480,Free,1,0
7,2147377152,2147377152,RO,0,4096,Committed,2,Private
8,2147381248,0,0,0,328007258112,Free,1,0
9,330154639360,330154639360,RW,0,24576,Reserved,0,Private


# Вывод

В результате лабораторной работы мной было выполнено исследование набора функций Win32 API, выполняющих операции с виртуальной памятью в Windows NT.